In [41]:
import os
# os.getcwd()
os.chdir('/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/src')
import sys
sys.path.append("../") # go to parent dir

from src.wind_stf.pipelines.data_science.nodes import scale_timeseries, _get_last_train_idx
from src.wind_stf.pipelines.data_engineering.nodes import build_power_centroids_mts

import pandas as pd
import pickle
from sklearn.preprocessing import QuantileTransformer

In [10]:
def unpickle(fp: str):
    with open(fp, 'rb') as handle:
        b = pickle.load(handle)
        
    return b

In [9]:
df_spatiotemporal = pd.read_hdf('../data/05_model_input/df_spatiotemporal.hdf')
cv_splits_dict = unpickle('../data/05_model_input/cv_splits_dict.pkl/2020-09-25T01.03.57.441Z/cv_splits_dict.pkl')
target_timeseries = ['DEF0C', 'DE111']

In [18]:
scale_timeseries(
        df_spatiotemporal,
        cv_splits_dict,
        target_timeseries)

AttributeError: 'numpy.float64' object has no attribute 'abs'

In [23]:
tss = df_spatiotemporal['temporal']

last_train_idx = _get_last_train_idx(tss, cv_splits_dict)
tss_train = tss.loc[:last_train_idx, target_timeseries]

quantile_transformer = QuantileTransformer(
    output_distribution='normal',
    random_state=0
)

quantile_transformer.fit( tss_train )

level_offset = abs( quantile_transformer.transform(
    tss_train[target_timeseries]
).min(axis=0) )

tss_scaled = pd.DataFrame(
    index=tss.index,
    columns=target_timeseries,
    data=quantile_transformer.transform(tss[target_timeseries]) + level_offset
)


In [55]:

tss_scaled

,DEF0C,DE111
2013-01-01,5.602990,6.731783
2013-01-02,5.526329,5.286310
2013-01-03,6.480263,5.526329
2013-01-04,6.617252,6.414190
2013-01-05,4.892643,6.397065
...,...,...
2015-12-27,6.809781,5.199008
2015-12-28,5.657192,4.267920
2015-12-29,6.602635,3.181828
2015-12-30,6.211051,4.585615


In [71]:
df_spatiotemporal.columns.levels

FrozenList([['spatial', 'temporal'], ['DE111', 'DE114', 'DE115', 'DE116', 'DE118', 'DE119', 'DE11A', 'DE11B', 'DE11C', 'DE11D', 'DE122', 'DE127', 'DE12A', 'DE12C', 'DE131', 'DE132', 'DE133', 'DE134', 'DE135', 'DE136', 'DE137', 'DE139', 'DE141', 'DE143', 'DE145', 'DE146', 'DE148', 'DE149', 'DE212', 'DE217', 'DE218', 'DE219', 'DE21B', 'DE21C', 'DE21E', 'DE21I', 'DE21J', 'DE21M', 'DE21N', 'DE226', 'DE227', 'DE228', 'DE22A', 'DE22C', 'DE232', 'DE234', 'DE235', 'DE236', 'DE237', 'DE238', 'DE23A', 'DE245', 'DE246', 'DE247', 'DE248', 'DE249', 'DE24A', 'DE24B', 'DE24D', 'DE251', 'DE256', 'DE258', 'DE259', 'DE25A', 'DE25B', 'DE25C', 'DE265', 'DE267', 'DE268', 'DE269', 'DE26A', 'DE26B', 'DE26C', 'DE275', 'DE276', 'DE277', 'DE278', 'DE279', 'DE27B', 'DE27C', 'DE27D', 'DE27E', 'DE401', 'DE402', 'DE405', 'DE406', 'DE407', 'DE408', 'DE40A', 'DE40B', 'DE40D', 'DE40E', 'DE40F', 'DE40G', 'DE40H', 'DE40I', 'DE712', 'DE716', 'DE718', 'DE719', ...], ['lat', 'lon', 'power']])

In [70]:
df_spatiotemporal['temporal'].columns.levels

FrozenList([['DE111', 'DE114', 'DE115', 'DE116', 'DE118', 'DE119', 'DE11A', 'DE11B', 'DE11C', 'DE11D', 'DE122', 'DE127', 'DE12A', 'DE12C', 'DE131', 'DE132', 'DE133', 'DE134', 'DE135', 'DE136', 'DE137', 'DE139', 'DE141', 'DE143', 'DE145', 'DE146', 'DE148', 'DE149', 'DE212', 'DE217', 'DE218', 'DE219', 'DE21B', 'DE21C', 'DE21E', 'DE21I', 'DE21J', 'DE21M', 'DE21N', 'DE226', 'DE227', 'DE228', 'DE22A', 'DE22C', 'DE232', 'DE234', 'DE235', 'DE236', 'DE237', 'DE238', 'DE23A', 'DE245', 'DE246', 'DE247', 'DE248', 'DE249', 'DE24A', 'DE24B', 'DE24D', 'DE251', 'DE256', 'DE258', 'DE259', 'DE25A', 'DE25B', 'DE25C', 'DE265', 'DE267', 'DE268', 'DE269', 'DE26A', 'DE26B', 'DE26C', 'DE275', 'DE276', 'DE277', 'DE278', 'DE279', 'DE27B', 'DE27C', 'DE27D', 'DE27E', 'DE401', 'DE402', 'DE405', 'DE406', 'DE407', 'DE408', 'DE40A', 'DE40B', 'DE40D', 'DE40E', 'DE40F', 'DE40G', 'DE40H', 'DE40I', 'DE712', 'DE716', 'DE718', 'DE719', ...], ['lat', 'lon', 'power']])

In [72]:
df_spatiotemporal['temporal']['power']

KeyError: 'power'

In [76]:
df_spatiotemporal['temporal'].columns.swaplevel(0, 1)

MultiIndex([('power', 'DE111'),
            ('power', 'DE114'),
            ('power', 'DE115'),
            ('power', 'DE116'),
            ('power', 'DE118'),
            ('power', 'DE119'),
            ('power', 'DE11A'),
            ('power', 'DE11B'),
            ('power', 'DE11C'),
            ('power', 'DE11D'),
            ...
            ('power', 'DEG0E'),
            ('power', 'DEG0F'),
            ('power', 'DEG0G'),
            ('power', 'DEG0I'),
            ('power', 'DEG0J'),
            ('power', 'DEG0K'),
            ('power', 'DEG0L'),
            ('power', 'DEG0M'),
            ('power', 'DEG0N'),
            ('power', 'DEG0P')],
           names=['var', 'district'], length=292)

In [67]:
df_spatiotemporal['temporal'].update( tss_scaled )

df_spatiotemporal['temporal'][['DEF0C', 'DE111']]

district,DEF0C,DE111
var,power,power
2013-01-01,NaN,NaN
2013-01-02,NaN,NaN
2013-01-03,NaN,NaN
2013-01-04,NaN,NaN
2013-01-05,NaN,NaN
...,...,...
2015-12-27,NaN,NaN
2015-12-28,NaN,NaN
2015-12-29,NaN,NaN


In [66]:
df_spatiotemporal['temporal'].size - df_spatiotemporal['temporal'].isna().sum().sum()

0

In [50]:
df_spatiotemporal_preprocessed = pd.read_hdf('../data/05_model_input/df_spatiotemporal_preprocessed.hdf')

In [56]:
df_spatiotemporal_preprocessed['temporal']['DEF0C']

var,power
2013-01-01,NaN
2013-01-02,NaN
2013-01-03,NaN
2013-01-04,NaN
2013-01-05,NaN
...,...
2015-12-27,NaN
2015-12-28,NaN
2015-12-29,NaN
2015-12-30,NaN


In [52]:
cv_splits_dict

{'pass_1': {'train_idx': [0, 730], 'test_idx': [730, 737]},
 'pass_2': {'train_idx': [0, 820], 'test_idx': [820, 827]},
 'pass_3': {'train_idx': [0, 910], 'test_idx': [910, 917]}}

In [3]:
df_spatial = pd.read_hdf('../data/04_feature/power-centroids-positions-2000-2015.hdf')

In [4]:
df_spatial

nuts_id                 DEF0C                DEF08                 DEA43  \
coords                    lat       lon        lat        lon        lat   
commissioning_date                                                         
1990-01-11          54.592251  9.398496  54.152187  10.523799  52.145711   
1990-01-12          54.592251  9.398496  54.152187  10.523799  52.145711   
1990-01-13          54.592251  9.398496  54.152187  10.523799  52.145711   
1990-01-14          54.592251  9.398496  54.152187  10.523799  52.145711   
1990-01-15          54.592251  9.398496  54.152187  10.523799  52.145711   
...                       ...       ...        ...        ...        ...   
2017-11-27          54.668452  9.308232  54.284751  10.959166  52.170185   
2017-11-28          54.668452  9.308232  54.284751  10.959166  52.170185   
2017-11-29          54.668452  9.308232  54.284751  10.959166  52.170185   
2017-11-30          54.668452  9.308232  54.284751  10.959166  52.170185   
2017-12-01          54.668452  9.308232  54.284751  10.959166  52.170185   

nuts_id                           DEF07                DEA5B            ...  \
coords                   lon        lat       lon        lat       lon  ...   
commissioning_date                                                      ...   
1990-01-11          8.726981  54.465024  8.864279  51.489703  7.946714  ...   
1990-01-12          8.726981  54.465024  8.864279  51.489703  7.946714  ...   
1990-01-13          8.726981  54.465024  8.864279  51.489703  7.946714  ...   
1990-01-14          8.726981  54.465024  8.864279  51.489703  7.946714  ...   
1990-01-15          8.726981  54.465024  8.864279  51.489703  7.946714  ...   
...                      ...        ...       ...        ...       ...  ...   
2017-11-27          8.675836  54.655649  8.977690  51.541023  8.170199  ...   
2017-11-28          8.675836  54.655649  8.977690  51.541023  8.170199  ...   
2017-11-29          8.675836  54.655649  8.977690  51.541023  8.170199  ...   
2017-11-30          8.675836  54.655649  8.977690  51.541023  8.170199  ...   
2017-12-01          8.675836  54.655649  8.977690  51.541023  8.170199  ...   

nuts_id                 DEA2C                DE221                 DE239  \
coords                    lat       lon        lat        lon        lat   
commissioning_date                                                         
1990-01-11          50.657977  6.865151  48.533997  12.208994  49.526011   
1990-01-12          50.657977  6.865151  48.533997  12.208994  49.526011   
1990-01-13          50.657977  6.865151  48.533997  12.208994  49.526011   
1990-01-14          50.657977  6.865151  48.533997  12.208994  49.526011   
1990-01-15          50.657977  6.865151  48.533997  12.208994  49.526011   
...                       ...       ...        ...        ...        ...   
2017-11-27          50.657977  6.865151  48.533997  12.208994  49.473041   
2017-11-28          50.657977  6.865151  48.533997  12.208994  49.473041   
2017-11-29          50.657977  6.865151  48.533997  12.208994  49.473041   
2017-11-30          50.657977  6.865151  48.533997  12.208994  49.473041   
2017-12-01          50.657977  6.865151  48.533997  12.208994  49.473041   

nuts_id                            DE21L                 DE715            
coords                    lon        lat        lon        lat       lon  
commissioning_date                                                        
1990-01-11          12.259649  47.985235  11.406016  49.454318  8.836523  
1990-01-12          12.259649  47.985235  11.406016  49.454318  8.836523  
1990-01-13          12.259649  47.985235  11.406016  49.454318  8.836523  
1990-01-14          12.259649  47.985235  11.406016  49.454318  8.836523  
1990-01-15          12.259649  47.985235  11.406016  49.454318  8.836523  
...                       ...        ...        ...        ...       ...  
2017-11-27          12.292913  47.982994  11.408302  49.455124  8.842887  
2017-11-28   

In [34]:
districts = list( df_spatial.columns.get_level_values(level=0) )

In [35]:
pd.DataFrame(
    data={d: list(map(tuple, df_spatial[d][['lat', 'lon']].values)) for d in districts},
    index=df_spatial.index
)

,DEF0C,DEF08,DEA43,DEF07,DEA5B,DEA34,DEF0B,DEE0E,DE40F,DEF05,...,DE21C,DE251,DE943,DE24C,DE266,DEA2C,DE221,DE239,DE21L,DE715
commissioning_date,,,,,,,,,,,,,,,,,,,,,
1990-01-11,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-12,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-13,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-14,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-15,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-27,"(54.66845166920903, 9.308231691788524)","(54.28475111816697, 10.959165956487873)","(52.170185478656535, 8.675836018274389)","(54.65564851845918, 8.977690293133016)","(51.541022924164615, 8.170199496974956)","(51.9732678848254, 6.938581

In [38]:
import numpy as np

np.mean[(1,2), (3,4)]

TypeError: 'function' object is not subscriptable

In [47]:
load_args = {
    'sep': ';',
    'header': 0, # for renaming columns with "names" below
    'names': ['id', 'power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat', 'N1'],
    'usecols': ['power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat'],
    'parse_dates': ['commissioning_date'],
    'infer_datetime_format': True,
}

sensors = pd.read_csv('../data/01_raw/metadata/wind_turbine_data.csv', **load_args)
build_power_centroids_mts(sensors)

,DEF0C,DEF08,DEA43,DEF07,DEA5B,DEA34,DEF0B,DEE0E,DE40F,DEF05,...,DE21C,DE251,DE943,DE24C,DE266,DEA2C,DE221,DE239,DE21L,DE715
commissioning_date,,,,,,,,,,,,,,,,,,,,,
1990-01-11,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-12,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-13,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-14,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
1990-01-15,"(54.59225069, 9.39849619)","(54.15218663, 10.52379932)","(52.14571144, 8.72698057)","(54.465024490000005, 8.864279046666667)","(51.48970308, 7.9467143)","(51.88482089, 6.841410819999999)","(54.13572501, 9.59636556)","(51.77738028, 12.71091294)","(53.28495298, 12.03897027)","(54.07802194, 9.03362531)",...,"(48.19447837999999, 11.182315500000001)","(49.239891435000004, 10.635932960000002)","(53.18951503333333, 8.26799569)","(50.04652981666666, 11.289874356666667)","(50.238105375, 10.37211452)","(50.65797713333333, 6.8651511266666665)","(48.53399724, 12.208993649999998)","(49.526011440000005, 12.259648975)","(47.98523547333334, 11.406015853333335)","(49.45431802000001, 8.83652282)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-27,"(54.66845166920903, 9.308231691788524)","(54.28475111816697, 10.959165956487873)","(52.170185478656535, 8.675836018274389)","(54.65564851845918, 8.977690293133016)","(51.541022924164615, 8.170199496974956)","(51.9732678848254, 6.938581

In [49]:
df_spatial = _

In [53]:
df_spatial['DEF0C']

commissioning_date
1990-01-11                 (54.59225069, 9.39849619)
1990-01-12                 (54.59225069, 9.39849619)
1990-01-13                 (54.59225069, 9.39849619)
1990-01-14                 (54.59225069, 9.39849619)
1990-01-15                 (54.59225069, 9.39849619)
                               ...                  
2017-11-27    (54.66845166920903, 9.308231691788524)
2017-11-28    (54.66845166920903, 9.308231691788524)
2017-11-29    (54.66845166920903, 9.308231691788524)
2017-11-30    (54.66845166920903, 9.308231691788524)
2017-12-01    (54.66845166920903, 9.308231691788524)
Freq: D, Name: DEF0C, Length: 10187, dtype: object